# Memory Leakage

In [ ]:
import gc

import torch


def memReport():
    cnt = 0
    for obj in gc.get_objects():
        if torch.is_tensor(obj):
            obj: torch.Tensor
            print(type(obj), obj.size(), obj.device, obj.name)
            cnt += 1
    print("total number of tensors: ", cnt)

In [ ]:
id()

In [ ]:
memReport()

In [ ]:
import os

import psutil


def cpuStats():
    psutil.virtual_memory()
    pid = os.getpid()
    py = psutil.Process(pid)
    memoryUse = py.memory_info()[0] / 2.0**20
    print("memory in MB:", memoryUse)

In [ ]:
torch.cuda.memory_usage()

In [ ]:
cpuStats()

In [ ]:
%%capture
%run -i "../src/train.py" "experiment=tinyset" "trainer=gpu" "trainer.max_epochs=2" "trainer.limit_train_batches=0.03" "trainer.limit_val_batches=0.03"

In [ ]:
# For comparison
%run -i "../src/train.py" "debug=fdr" "model=mnist_ann" "trainer.fast_dev_run=100"

In [ ]:
memReport()

In [ ]:
cpuStats()

In [ ]:
gc.collect()

In [ ]:
memReport()

In [ ]:
cpuStats()

In [ ]:
print(torch.cuda.memory_summary())

# grad norm track

In [ ]:
# log second gradient norm of the model
%run -i "../src/train.py" "+trainer.limit_train_batches=0.2" "+trainer.limit_val_batches=0.2" "+trainer.limit_test_batches=0.2" "+trainer.track_grad_norm=2"

# Custom profiler

Note) pytorch lightning provides profiler.
see below links
- https://lightning.ai/docs/pytorch/stable/tuning/profiler_basic.html
- https://lightning.ai/docs/pytorch/stable/tuning/profiler_intermediate.html

In [ ]:
import torch  # noqa
from lightning.pytorch.profilers import SimpleProfiler

prof = SimpleProfiler()


def foo(x, y):
    with prof.profile("foo"):
        return x + y

In [ ]:
foo(torch.Tensor([1, 2]), torch.Tensor([3, 4]))

In [ ]:
print(prof.summary())

# Import Time Profiler


In [ ]:
import cProfile
import io
import pstats


def import_module():

    from src.core.eqprop.strategy import NewtonStrategy


pr = cProfile.Profile()
pr.enable()
import_module()
pr.disable()

s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats("cumulative")
ps.print_stats()

print(s.getvalue())